In [1]:
#!pip install pql
import pql

In [2]:
from optimade.filter import Parser
from lark import Transformer
p = Parser(version=(0, 9, 6))

In [3]:
import pql
from optimade.filter import Parser
from lark import Transformer


optiMadeToPQLOperatorSwitch = {
    "=":"==",
    "<=":"<=",
    ">=":">=",
    "=>":">=",
    "!=": "!=",
    "<":"<",
    ">":">",
}

def OptiMadeToPQLOperatorValidator(x):
    return optiMadeToPQLOperatorSwitch[x]

def combineMultiple(string, index):
    firstQuoteIndex = 0
    for firstQuoteIndex in reversed(range(0, index)):
        if(ord(string[firstQuoteIndex]) == 34):
            break
    lastQuoteIndex = index
    for lastQuoteIndex in range(index, len(string)):
        if(ord(string[lastQuoteIndex])== 34):
            break
    insertion = string[firstQuoteIndex+1:lastQuoteIndex]
    insertion = insertion.split(",")
    return string[:firstQuoteIndex] + 'all({})'.format(insertion) + string[lastQuoteIndex+1:]

def parseInput(PQL):
    length = len(PQL)
    i = 0
    while(i < length):
        if(PQL[i] == ","):
            PQL = combineMultiple(PQL, i)
        i = i + 1 
    return PQL

class OptimadeToPQLTransformer(Transformer):
    def comparison(self, args):
        A = str(args[0])
        B = str(args[2])
        operator = OptiMadeToPQLOperatorValidator(args[1])
#         print('"' + A +'"' + operator + '"' + B + '"')
#         return '"' + A +'"' + operator + '"' + B + '"'
        return A + operator + '"' + B + '"'
    def atom(self, args):
#         print("atom:", args, len(args))
        return args[0]
    def term(self, args):
#         print("term:", args, len(args))
        result = ""
        for arg in args:
            if arg.lower() == "and" or arg.lower() == "or":
                arg = arg.lower()
            result = result + " " + arg.lower()
        return "(" + result.lstrip() + ")"
    
    def expression(self, args):
#         print("expression:",args)
        result = ""
        for arg in args:
            lower = arg.lower()
            if(lower == "and" or lower == "or"):
                arg = lower
            result = result + " " + arg
        return result.lstrip()
    def start(self, args):
        return args[1]

def optimadeToMongoDBConverter(optimadeQuery):
    p = Parser(version=(0, 9, 6))
#     print("optimadeQuery:", optimadeQuery)
#     print()
    tree = p.parse(optimadeQuery)
#     print("tree:\n", tree)
#     print()
    rawPQL = OptimadeToPQLTransformer().transform(tree)
    cleanedPQL = parseInput(rawPQL)
    print("cleanedPQL:", cleanedPQL)
    print()
    rawMongoDbQuery = {}
    try:
        rawMongoDbQuery = pql.find(cleanedPQL)
    except:
        return "ERROR: Unable to execute conversion"
    
    _cleanMongo(rawMongoDbQuery)
    return cleanedMongoDbQuery
    

In [30]:
def cleanMongo(rawMongoDbQuery):
    for k in rawMongoDbQuery:
        value = rawMongoDbQuery[k]
        if(type(value) == list):
            for v in value:
                cleanMongo(v)
        elif(type(value) == dict):
            cleanMongo(value)
        elif(type(value) == str):
            try:
                value = float(value)
                print(float(value))
            except:
                f = value
            
        else:
            print("value", value)
            



In [31]:
x = {'$and': [{'a': {'$lt': '1'}}, {'$or': [{'b': {'$gt': '2'}}, {'c': '3'}]}]}
cleanMongo(x)

1.0
2.0
3.0


In [32]:
x

{'$and': [{'a': {'$lt': '1'}}, {'$or': [{'b': {'$gt': '2'}}, {'c': '3'}]}]}

In [ ]:
print(optimadeToMongoDBConverter('filter=A<1'))

print(optimadeToMongoDBConverter('filter=A<1 AnD (B > 2 or C = 3)'))

In [26]:
query = "delta_volume=-89.79884478160747 and id=2"
optimadeToMongoDBConverter("filter="+ query)

cleanedPQL: ((delta_volume=="-89.79884478160747") and id=="2")



{'$and': [{'delta_volume': '-89.79884478160747'}, {'id': '2'}]}

this works: <br>
materials_collection.find_one({'density':{'$gt':float('3')}})

In [10]:
rawPQL = 'd<1 and (a<0 or c >2)'


In [12]:
import unittest
class ConverterTest(unittest.TestCase):
    def setup(self):
        pass
    def testAll(self):
        self.test_one_input()
        self.test_two_inputs_with_and()
        self.test_two_inputs_with_or()
        self.test_valid_numbers_positive()
        
    def test_one_input(self):
        testCase = "filter=a<0" 
        answer = {'a': {'$lt': 0}}
        message = "test_one_input failed"
        self.assertEqual(optimadeToMongoDBConverter(testCase), answer, message)
        print("passed: test_one_input passed")
    def test_two_inputs_with_and(self):
        testCase = "filter=a<0 and b>2" 
        answer = {'$and': [{'a': {'$lt': 0}}, {'b': {'$gt': 2}}]}
        message = "test_two_inputs_with_and failed"
        self.assertEqual(optimadeToMongoDBConverter(testCase), answer, message)
        print("passed: test_two_inputs_with_and")
    def test_two_inputs_with_or(self):
        testCase = "filter=a<0 or b>2" 
        answer = {'$or': [{'a': {'$lt': 0}}, {'b': {'$gt': 2}}]}
        message = "test_two_inputs_with_or failed"
        self.assertEqual(optimadeToMongoDBConverter(testCase), answer, message)
        print("passed: test_two_inputs_with_or")
    def test_valid_numbers_with_positive_sign(self):
        testCase = "filter= a=+1" 
        answer = {'a': +1}
        message = "test_valid_numbers_positive failed"
        self.assertEqual(optimadeToMongoDBConverter(testCase), answer, message)
        print("passed: test_valid_numbers_positive")

In [13]:
ConverterTest().testAll()

optimadeQuery: filter=a<0

cleanedPQL: (a<0)

passed: test_one_input passed
optimadeQuery: filter=a<0 and b>2

cleanedPQL: ((a<0) and b>2)

passed: test_two_inputs_with_and
optimadeQuery: filter=a<0 or b>2

cleanedPQL: ((a<0) or b>2)

passed: test_two_inputs_with_or


AttributeError: 'ConverterTest' object has no attribute 'test_valid_numbers_positive'

In [14]:
import pql
from optimade.filter import Parser
from lark import Transformer

p = Parser(version=(0, 9, 6))
x = p.parse('filter= A<1 and b < 2')
print(x.pretty())

start
  filter=
  expression
    term
      term
        atom
          comparison
            A
            <
            1
      and
      atom
        comparison
          b
          <
          2



In [15]:
import pql
from optimade.filter import Parser
from lark import Transformer

p = Parser(version=(0, 9, 6))
print(p.parse('filter= (A<1 or b < 2) and c < 1').pretty())

start
  filter=
  expression
    term
      term
        term
          term
            atom
              comparison
                A
                <
                1
          or
          atom
            comparison
              b
              <
              2
        and
        atom
          comparison
            c
            <
            1



In [16]:
print(p.parse('filter= A<1 or (b < 2 and c < 1)').pretty())

start
  filter=
  expression
    expression
      term
        atom
          comparison
            A
            <
            1
    or
    term
      term
        term
          atom
            comparison
              b
              <
              2
        and
        atom
          comparison
            c
            <
            1



In [17]:
p.parse('filter= A<1 or (b < 2 and c < 1)') == p.parse('filter= (A<1 or b < 2) and c < 1')

False